<a href="https://colab.research.google.com/github/shris2810/link_prediction/blob/with_lib/countries_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/shris2810/link_prediction.git
%cd link_prediction

Cloning into 'link_prediction'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
/content/link_prediction


In [5]:
!pip install torch_geometric
!pip install optuna


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.9/413.9 kB 14.7 MB/s eta 0:00:00


In [7]:
import torch
from torch_geometric.datasets import KarateClub
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from torch.nn import Linear
from torch_geometric.nn import GCNConv

In [8]:
#Load the dataset
dataset = KarateClub()
data = dataset[0]

In [9]:
print(dataset.num_features)

34


In [12]:
import pandas as pd
# node features to df
node_features_df = pd.DataFrame(data.x.numpy(), columns=[f'features_{i}' for i in range(34)])
node_features_df['node'] = node_features_df.index
node_features_df.set_index('node', inplace=True)

edge_index_df = pd.DataFrame(data.edge_index.numpy().T, columns=['source','target'])

labels_df = pd.DataFrame(data.y.numpy(), columns=['label'])
labels_df['node'] = labels_df.index
labels_df.set_index('node', inplace=True)

In [16]:
print(f'number of nodes : {data.num_nodes}')
print(f'number of edges : {data.edge_index.shape[1]}')

print(f"node features : {node_features_df.shape}")
print(node_features_df.head())

print(f"edges : {edge_index_df.shape}")
print(edge_index_df.head())

print(f"labels : {labels_df.shape}")
print(labels_df.head())

number of nodes : 34
number of edges : 156
node features : (34, 34)
      features_0  features_1  features_2  features_3  features_4  features_5  \
node                                                                           
0            1.0         0.0         0.0         0.0         0.0         0.0   
1            0.0         1.0         0.0         0.0         0.0         0.0   
2            0.0         0.0         1.0         0.0         0.0         0.0   
3            0.0         0.0         0.0         1.0         0.0         0.0   
4            0.0         0.0         0.0         0.0         1.0         0.0   

      features_6  features_7  features_8  features_9  ...  features_24  \
node                                                  ...                
0            0.0         0.0         0.0         0.0  ...          0.0   
1            0.0         0.0         0.0         0.0  ...          0.0   
2            0.0         0.0         0.0         0.0  ...          0.0   
3

In [17]:
#assigning countires to each student
num_countries = 4

np.random.seed(42)
countries = torch.tensor(np.random.choice(num_countries, data.num_nodes))

data.y = countries

In [18]:
print(data)
print(data.x)
print(data.edge_index)
print(data.y)

Data(x=[34, 34], edge_index=[2, 156], y=[34], train_mask=[34])
tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,
          3,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,
          7,  7,  8,  8,  8,  8,  8,  9,  9, 10, 10, 10, 11, 12, 12, 13, 13, 13,
         13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 19, 20, 20, 21,
         21, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26, 27, 27,
         27, 27, 28, 28, 28, 29, 29, 29, 29, 30, 30, 30, 30, 31, 31, 31, 31, 31,
         31, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33,
         33, 33, 33, 33, 33, 33, 33

In [20]:
#only one layer so don't get info of neigbour of neigbour
#define model

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.gcn = GCNConv(dataset.num_features,3)
    self.out = Linear(3, num_countries)

  def forward(self, x, edge_index):
    h = self.gcn(x, edge_index).relu()
    z = self.out(h)
    return h, z

model = GCN()
print(model)

GCN(
  (gcn): GCNConv(34, 3)
  (out): Linear(in_features=3, out_features=4, bias=True)
)


In [22]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.02)

#calculate accuracy
def accuracy(pred_y, y):
  return (pred_y == y ).sum()/len(y)

In [24]:
# save model
embeddings = []
losses = []
accuracies = []
outputs = []

# Training loop
for epoch in range(200):

  optimizer.zero_grad()
  h, z = model(data.x, data.edge_index)
  loss = criterion(z, data.y)
  acc = accuracy(z.argmax(dim=1), data.y)

  loss.backward()
  optimizer.step()

  embeddings.append(h)
  losses.append(loss)
  accuracies.append(acc)
  outputs.append(z.argmax(dim=1))

  if epoch % 10 == 0:
    print(f'Epoch {epoch:>3} | Loss {loss:.2f} | Acc {acc*100:.2f}')

Epoch   0 | Loss 1.48 | Acc 20.59
Epoch  10 | Loss 1.37 | Acc 20.59
Epoch  20 | Loss 1.22 | Acc 61.76
Epoch  30 | Loss 1.04 | Acc 61.76
Epoch  40 | Loss 0.87 | Acc 67.65
Epoch  50 | Loss 0.72 | Acc 79.41
Epoch  60 | Loss 0.59 | Acc 85.29
Epoch  70 | Loss 0.47 | Acc 88.24
Epoch  80 | Loss 0.38 | Acc 94.12
Epoch  90 | Loss 0.30 | Acc 97.06
Epoch 100 | Loss 0.23 | Acc 100.00
Epoch 110 | Loss 0.18 | Acc 100.00
Epoch 120 | Loss 0.14 | Acc 100.00
Epoch 130 | Loss 0.11 | Acc 100.00
Epoch 140 | Loss 0.09 | Acc 100.00
Epoch 150 | Loss 0.07 | Acc 100.00
Epoch 160 | Loss 0.06 | Acc 100.00
Epoch 170 | Loss 0.05 | Acc 100.00
Epoch 180 | Loss 0.04 | Acc 100.00
Epoch 190 | Loss 0.04 | Acc 100.00


In [44]:
!git fetch

In [40]:
!git checkout -b with_lib origin/with_lib

fatal: A branch named 'with_lib' already exists.


In [41]:
!git config --global user.name "Srishti Malik"
!git config --global user.email "shris2810@gmail.com"


In [45]:
!git add .
!git commit -m "link prediction model code with libraries"
!git push origin with_lib


On branch with_lib
Your branch is up to date with 'origin/with_lib'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [48]:
!ls -la




total 16
drwxr-xr-x 3 root root 4096 Feb  3 16:26 .
drwxr-xr-x 1 root root 4096 Feb  3 16:26 ..
drwxr-xr-x 8 root root 4096 Feb  3 17:10 .git
-rw-r--r-- 1 root root   17 Feb  3 16:26 README.md
